In [ ]:
#*****************************************************************************************************
# Fix settings concerning the ansatz, optimisation and backend
#*****************************************************************************************************
from time import time 
import pickle 
import numpy as np

from discopy import Ty, Id, Box, Diagram, Word
from discopy.rigid import Cup, Cap, Functor, Swap
from discopy.quantum.circuit import bit, qubit
from discopy.quantum import Measure
from discopy.quantum.tk import to_tk
from discopy.quantum.tk import Circuit as tk_Circuit_inDCP

from pytket import Circuit as tk_Circuit

#-----------------------------
# atomic pregroup grammar types
#-----------------------------
s, n = Ty('S'), Ty('N')

#----------------------------------------
# settings concerning the ansaetze
#----------------------------------------
q_s = 0        # number of qubits for sentence type s
q_n = 1        # number of qubits for noun type n
depth = 2      # number of IQP layers for non-single-qubit words
p_n = 1        # number of parameters for a single-qubit word (noun); valued in {1,2,3}.

#----------------------------------------
# Parameters concerning the optimisation
#----------------------------------------
n_runs = 20       # number of runs over training procedure
niter  = 2000     # number of iterations for any optimisation run of training.


In [ ]:
#******************************************
# Data import
#******************************************

# import train and test datasets: the data entries are all strings of the form 'label sentence' 
# with the label in {0,1} and with the sentence of the form "word1_POStag1 word2_POStag2 ..."

with open('../datasets/rp_train_data.txt') as f:
    training_data_raw = f.readlines()

with open('../datasets/rp_test_data.txt') as f:
    testing_data_raw = f.readlines()

In [ ]:
#***************************************************************
# Turn the raw input data into data structures convenient below
#***************************************************************

vocab_dict = dict()     # dictionary to be filled with the vocabulary in the form { word : POStag }
data = dict()           # dictionary to be filled with all the data (from train and test subsets); entries of the 
                        # form { sentence : label } with label encoding '1' as [1.0, 0.0] and '0' as [0.0, 1.0].
training_data = []      # list of sentences in the train dataset as strings "word1 word2 ..."
testing_data = []       # list of sentences in the test dataset as strings "word1 word2 ..."

# Go through the train data
for sent in training_data_raw:
    words = sent[2:].split() 
    sent_untagged = ''
    for word in words:
        word_untagged, tag = word.split('_')
        vocab_dict[word_untagged] = tag
        sent_untagged += word_untagged + ' '
    sentence = sent_untagged[:-1]
    training_data.append(sentence)
    label = [1.0, 0.0] if sent[0] == '1' else [0.0, 1.0]
    data[sentence] = label

# Go through the test data
for sent in testing_data_raw:
    words = sent[2:].split() 
    sent_untagged = ''
    for word in words:
        word_untagged, tag = word.split('_')
        vocab_dict[word_untagged] = tag
        sent_untagged += word_untagged + ' '
    sentence = sent_untagged[:-1]
    testing_data.append(sentence)
    label = [1.0, 0.0] if sent[0] == '1' else [0.0, 1.0]
    data[sentence] = label

In [ ]:
#******************************************
# Pregroup grammar level
#******************************************

# The needed composite types of words
tv = n.r @ s @ n.l                          # type of transitive verbs
that_subj_type = n.r @ n @ s.l @ n          # type of a relative pronoun as a subject
that_obj_type = n.r @ n @ (n.l).l @ s.l     # type of a relative pronoun as an object

# The two needed pregroup grammar wirings 
#---------------------------------
grammar_subj = (Cup(n, n.r) @ Id(n) @ Id(s.l) @ Cup(n, n.r) @ Id(s) @ Cup(n.l, n)) >> (Id(n) @ Cup(s.l, s))
grammar_obj = (Cup(n, n.r) @ Id(n @ (n.l).l @ s.l) @ Cup(n, n.r) @ Id(s @ n.l)) >> (Id(n @ (n.l).l) @ Cup(s.l, s) @ Id(n.l)) >> (Id(n) @ Cup((n.l).l, n.l))

# The pregroup vocabulary 
#------------------------
# (note we need the noun effects with type n.l as well as n.r) 
vocab_dict_pg = dict()
vocab_dict_pg_psr = dict()
vocab_dict_pg_psr_nl = dict()
for word in vocab_dict:
    if vocab_dict[word] == 'N':
        vocab_dict_pg.update({word: Word(word, n)})
        vocab_dict_pg_psr.update({word: Box(word, n.r, Ty())})
        vocab_dict_pg_psr_nl.update({word: Box(word, n.l, Ty())})
    if vocab_dict[word] == 'V':
        vocab_dict_pg.update({word: Word(word, tv)})
        vocab_dict_pg_psr.update({word: Word(word, tv)})
vocab = list(vocab_dict_pg.values())
vocab_psr = list(vocab_dict_pg_psr.values())

# The corresponding two kinds of 'that'
that_subj = Word('that', that_subj_type)
that_obj = Word('that', that_obj_type)
        
# Go through data and create the diagrams for all phrases and fill into data_pg, 
# while filling data_pg_psr with the phrases where the nouns have been 'bent around'.
data_pg = dict()
data_pg_psr = dict()
for sentstr in data:
    word_list = sentstr.split()
    if data[sentstr] == [0.0, 1.0]:   # subject case
        sent = (vocab_dict_pg[word_list[0]] @ that_subj @ vocab_dict_pg[word_list[2]] @ vocab_dict_pg[word_list[3]]) >> grammar_subj
        sent_psr = (Cap(n.r, n) @ Cap(n, n.l)) >> (vocab_dict_pg_psr[word_list[0]] @ Id(n) @ that_subj @ vocab_dict_pg[word_list[2]] @ Id(n) @ vocab_dict_pg_psr_nl[word_list[3]]) >> grammar_subj
    if data[sentstr] == [1.0, 0.0]:   # object case
        sent = (vocab_dict_pg[word_list[0]] @ that_obj @ vocab_dict_pg[word_list[2]] @ vocab_dict_pg[word_list[3]]) >> grammar_obj
        sent_psr = (Cap(n.r, n) @ Cap(n.r, n)) >> (vocab_dict_pg_psr[word_list[0]] @ Id(n) @ that_obj @ vocab_dict_pg_psr[word_list[2]] @ Id(n) @ vocab_dict_pg[word_list[3]]) >> grammar_obj
    data_pg.update({sentstr: sent})
    data_pg_psr.update({sentstr: sent_psr.normal_form()})

In [ ]:
#************************************************************************
# Two example sentences to see the two different cases diagrammatically:
#************************************************************************

print('----------------------------------------------')
print('Ordinary DisCoCat diagrams')
print('----------------------------------------------')
print('This is the subject case ([0.0, 1.0]):')
data_pg['organization that establish church'].draw()
print('This is the object case ([1.0, 0.0]):')
data_pg['organization that church establish'].draw()
print('----------------------------------------------')
print('The DisCoCat diagrams with nouns bent around')
print('----------------------------------------------')
print('This is the subject case ([0.0, 1.0]):')
data_pg_psr['organization that establish church'].draw()
print('This is the object case ([1.0, 0.0]):')
data_pg_psr['organization that church establish'].draw()

In [ ]:
#*****************************************************
# Translation to quantum circuits
#*****************************************************
from discopy.quantum import Ket, IQPansatz, Bra
from discopy.quantum.gates import sqrt, H, CZ, Rz, Rx, CX
from discopy.quantum.circuit import Id
from discopy import CircuitFunctor
from discopy.quantum.circuit import Circuit as DCP_Circuit


ob = {s: q_s, n: q_n}                           # assignment of number of qubits to atomic grammatical types
ob_cqmap = {s: qubit ** q_s, n: qubit ** q_n}   # the form in which it is needed for discopy's cqmap module

#-----------------------------------------
# parametrised part of ansaetze
#-----------------------------------------

def single_qubit_iqp_ansatz(params):
    if len(params) == 1:
        return Rx(params[0])  
    if len(params) == 2:
        return Rx(params[0]) >> Rz(params[1])
    if len(params) == 3:
        return IQPansatz(1, params)       

def ansatz_state(state, params):  
    arity = sum(ob[Ty(factor.name)] for factor in state.cod)
    if arity == 1:
        return Ket(0) >> single_qubit_iqp_ansatz(params)
    else:
        return Ket(*tuple([0 for i in range(arity)])) >> IQPansatz(arity, params)
    
def ansatz_effect(effect, params):  
    arity = sum(ob[Ty(factor.name)] for factor in effect.dom)
    if arity == 1:
        return single_qubit_iqp_ansatz(params) >> Bra(0)
    else:
        return IQPansatz(arity, params) >> Bra(*tuple([0 for i in range(arity)]))
       
def ansatz(box, params):
    dom_type = box.dom
    cod_type = box.cod
    if len(dom_type) == 0 and len(cod_type) != 0:
        return ansatz_state(box, params)
    if len(dom_type) != 0 and len(cod_type) == 0:
        return ansatz_effect(box, params)

#-----------------------------------------
# unparametrised part of ansaetze
#-----------------------------------------

def relpron_ansatz(s_o_case):                     # relpron modelled as a frobenius spider in a state
    GHZ = Ket(0, 0, 0) >> H @ H @ H >> CZ @ Id(1) >> Id(1) @ CZ >> H @ Id(1) @ H
    plus_state = Ket(*[0 for i in range(ob[s])])  # on s type wire
    if ob[s] != 0:                                # for 1-dim space (0 qubits) there is only one normalised states.
        gate = H
        for i in range(1, ob[s]): 
            gate = gate @ H    
        plus_state = plus_state >> gate
    circ = GHZ
    for i in range(1, ob[n]):
        circ = circ @ GHZ
    circ = circ >> DCP_Circuit.permutation([x + 3*y for x in range(3) for y in range(ob[n])])
    if s_o_case == 'subj':
        return circ >> Id(ob_cqmap[n] @ ob_cqmap[n]) @ plus_state @ Id(ob_cqmap[n])
    if s_o_case == 'obj':
        return circ @ plus_state
    
#----------------------------------------------------------
# Define parametrised functor to quantum circuits
#----------------------------------------------------------

def F(params): 
    ar = dict()
    # parametrised morphisms
    for i in range(len(vocab_psr)):
        pgbox = vocab_psr[i]
        qbox = ansatz(vocab_psr[i], params[i])
        ar.update({pgbox: qbox})
        if pgbox.cod == Ty():
            ar.update({Box(pgbox.name, n.l, Ty()): qbox})
    # unparametrised morphisms
    ar.update({that_subj: relpron_ansatz('subj'), that_obj: relpron_ansatz('obj')})
    return CircuitFunctor(ob_cqmap, ar)

In [ ]:
#*****************************************************
# The functions to deal with the parametrisation
#*****************************************************

def param_shapes(vocab_psr):
    parshapes = []    
    for box in vocab_psr:
        dom_type = box.dom
        cod_type = box.cod
        dom_arity = sum(ob[Ty(factor.name)] for factor in box.dom)
        cod_arity = sum(ob[Ty(factor.name)] for factor in box.cod)
        if dom_arity == 0 or cod_arity == 0:  # states and effects
            arity = max(dom_arity, cod_arity)
            assert arity != 0
            if arity == 1:
                parshapes.append((p_n,))       
            if arity != 1:
                parshapes.append((depth, arity-1))
    return parshapes

def rand_params(par_shapes):
    params = np.array([]) 
    for i in range(len(par_shapes)):
         params = np.concatenate((params, np.ravel(np.random.rand(*par_shapes[i]))))
    return params 

def reshape_params(unshaped_pars, par_shapes):
    pars_reshaped = [[] for ii in range(len(par_shapes))]
    shift = 0
    for ss, s in enumerate(par_shapes):
        idx0 = 0 + shift
        if len(s) == 1:
            idx1 = s[0] + shift
        elif len(s) == 2:
            idx1 = s[0] * s[1] + shift
        pars_reshaped[ss] = np.reshape(unshaped_pars[idx0:idx1], s)
        if len(s) == 1:
            shift += s[0]
        elif len(s) == 2:
            shift += s[0] * s[1]
    return pars_reshaped

In [ ]:
#****************************************
# The parameters of the current model
#****************************************

par_shapes = param_shapes(vocab_psr)
rand_unshaped_pars = rand_params(par_shapes)
rand_shaped_pars = reshape_params(rand_unshaped_pars, par_shapes)

print('Number of parameters:    ', len(rand_unshaped_pars))

In [ ]:
#**************************************************************
# Quantum circuit diagrams for 'that' ansaetze and example sentences 
#**************************************************************

func = F(rand_shaped_pars)

print("The ansatz for 'that' in subject case:")
relpron_ansatz('subj').draw()

print("The ansatz for 'that' in object case:")
relpron_ansatz('obj').draw()

print('Example diagram from above for the subject case mapped under F:')
func(data_pg_psr['organization that establish church']).draw(draw_box_labels=True, figsize=(7, 7), nodesize = 0.4)

print('Example diagram from above for the object case mapped under F:')
func(data_pg_psr['organization that church establish']).draw(draw_box_labels=True, figsize=(7, 7), nodesize = 0.4)

In [ ]:
#********************************************************************************************
# Encode data such that the circuits (for one call of cost function etc.) can be sent as one
# job to quantum hardware.
#********************************************************************************************

train_labels = []
train_circuits_pg_psr = []
for sentstr in training_data:
    train_circuits_pg_psr.append(data_pg_psr[sentstr])
    train_labels.append(np.array(data[sentstr]))
train_labels = np.array(train_labels)

test_labels = []
test_circuits_pg_psr = []
for sentstr in testing_data:
    test_circuits_pg_psr.append(data_pg_psr[sentstr])
    test_labels.append(np.array(data[sentstr]))
test_labels = np.array(test_labels)

In [ ]:
#**********************************************************************************************************
# The cost function for optimisation and the error functions 
#**********************************************************************************************************
from jax import numpy as jnp
from jax import jit

def get_cost(unshaped_params):
    func = F(reshape_params(unshaped_params, par_shapes))
    train_circuits = [func(circ) for circ in train_circuits_pg_psr]
    results = DCP_Circuit.eval(*train_circuits)
    results_tweaked = [jnp.abs(jnp.array(res.array) - 1e-9) for res in results]
    pred_labels_distrs = [res / jnp.sum(res) for res in results_tweaked]
    cross_entropies = jnp.array([jnp.sum(train_labels[s] * jnp.log2(pred_labels_distrs[s])) for s in range(len(train_labels))])
    return -1 / len(training_data) * jnp.sum(cross_entropies)

def get_train_error(unshaped_params):
    func = F(reshape_params(unshaped_params, par_shapes))
    train_circuits = [func(circ) for circ in train_circuits_pg_psr]
    results = DCP_Circuit.eval(*train_circuits)
    results_tweaked = [jnp.abs(jnp.array(res.array) - 1e-9) for res in results]
    pred_labels_distrs = [res / jnp.sum(res) for res in results_tweaked]
    assert len(pred_labels_distrs[0]) == 2  # rounding only makes sense if labels are binary tuples
    pred_labels = [jnp.round(res) for res in pred_labels_distrs]
    error = 0.0
    for i in range(len(pred_labels)):
        diff = jnp.sum(jnp.abs(train_labels[i] - pred_labels[i]))
        error += jnp.min(jnp.array([diff, 1.0]))
    return error * 100 / len(training_data)

def get_test_error(unshaped_params):
    func = F(reshape_params(unshaped_params, par_shapes))
    test_circuits = [func(circ) for circ in test_circuits_pg_psr]
    results = DCP_Circuit.eval(*test_circuits)
    results_tweaked = [jnp.abs(jnp.array(res.array) - 1e-9) for res in results]
    pred_labels_distrs = [res / jnp.sum(res) for res in results_tweaked]
    assert len(pred_labels_distrs[0]) == 2  # rounding only makes sense if labels are binary tuples
    pred_labels = [jnp.round(res) for res in pred_labels_distrs]
    error = 0.0
    for i in range(len(pred_labels)):
        diff = jnp.sum(jnp.abs(test_labels[i] - pred_labels[i]))
        error += jnp.min(jnp.array([diff, 1.0]))
    return error * 100 / len(testing_data)

In [ ]:
#*************************************************************************************************
# Define the jitted versions of above three functions. Then one by one do a 
# loop over two calls to let jit do its thing so that function call is fast when doing optimisation.
#*************************************************************************************************

get_cost_jit = jit(get_cost)
get_train_error_jit = jit(get_train_error)
get_test_error_jit = jit(get_test_error)

In [ ]:
for i in range(2):
    rand_unshaped_pars = rand_params(par_shapes)
    print('-------------')
    start = time()
    print('Cost: ', get_cost_jit(rand_unshaped_pars))
    print('Time taken for this iteration: ', time()-start)

In [ ]:
for i in range(2):
    rand_unshaped_pars = rand_params(par_shapes)
    print('-------------')
    start = time()
    print('Train Error: ', get_train_error_jit(rand_unshaped_pars))
    print('Time taken for this iteration: ', time()-start)

In [ ]:
for i in range(2):
    rand_unshaped_pars = rand_params(par_shapes)
    print('-------------')
    start = time()
    print('Test Error : ', get_test_error_jit(rand_unshaped_pars))
    print('Time taken for this iteration: ', time()-start)

In [ ]:
#**********************************************************************************
# Minimization algorithm
#**********************************************************************************

# This is building on the minimizeSPSA function from the noisyopt package (https://github.com/andim/noisyopt);
# here only adjusted for our purposes (mostly with quantum implementations in mind)

def my_spsa(get_cost, get_train_error, x0,
            bounds=None, niter=100, a=1.0, c=1.0, alpha=0.602, gamma=0.101,
            print_iter=False, filename='spsa_output'):
    A = 0.01 * niter
    N = len(x0)
    if bounds is None:
        project = lambda x: x
    else:
        bounds = np.asarray(bounds)
        project = lambda x: np.clip(x, bounds[:, 0], bounds[:, 1])    
    param_history = []
    func_history = []
    error_history = []
    x = x0    
    
    # Loop over iterations
    for k in range(niter):
        if print_iter:
            print('-------------', '\n', 'iteration: ', k, sep='')
        start = time()
        
        # determine stepping parameters
        ak = a/(k+1.0+A)**alpha
        ck = c/(k+1.0)**gamma
        delta = np.random.choice([-1, 1], size=N)
        
        # move in + direction from previous x
        xplus = project(x + ck*delta)        
        if print_iter:
            print('Call for xplus')
        funcplus = get_cost(xplus)
        
        # move in - direction from previous x
        xminus = project(x - ck * delta)
        if print_iter:
            print('Call for xminus')
        funcminus = get_cost(xminus)
        
        # new step
        grad = (funcplus - funcminus) / (xplus-xminus)
        x = project(x - ak*grad)
        param_history.append(x)
        
        # determine current func and error
        current_func_value = get_cost(x)
        error = get_train_error(x)
        func_history.append(current_func_value)
        error_history.append(error)

        # save to file
        dump_data = {
            'param_history': param_history,
            'func_history': func_history,
            'error_history': error_history
        }
        with open(filename+'.pickle', 'wb') as file_handle:
            pickle.dump(dump_data, file_handle)
        
        if print_iter:
            print('Time taken for this iteration: ', time() - start)
    return param_history, func_history, error_history 

In [ ]:
#*****************************************************
# Optimisation settings
#*****************************************************

bounds = [[0.0, 1.0] for ii in range(len(rand_unshaped_pars))]

c_fix = 0.1 
#leave alpha and gamma as the default (which is as recommended), i.e. alpha = 0.602 and gamma = 0.101

#-----------------------------------
# calculate well-educated guess for parameter 'a'. 
# (below calcucation follows the heuristics from: 
#  https://www.jhuapl.edu/SPSA/PDF-SPSA/Spall_Implementation_of_the_Simultaneous.PDF )
#-----------------------------------

desired_param_change = 0.005  # rough change of a parameter in early iterations.

alpha = 0.602     
A = niter*0.01    
c_0 = c_fix

a_est = 0.0
nruns = 1000
for l in range(nruns):
    rand_unshaped_pars = rand_params(par_shapes)
    delta_0 = np.array([np.random.randint(0,2) for i in range(len(rand_unshaped_pars))])*2\
              - np.array([1 for i in range(len(rand_unshaped_pars))])
    g0_estimate = (get_cost_jit(rand_unshaped_pars + c_0*delta_0) - get_cost_jit(rand_unshaped_pars - c_0*delta_0))/(2*c_0)
    a_est += np.abs(desired_param_change * ((A +1)**alpha) / g0_estimate )
a_est = a_est/nruns
print('Calculated good choice for a=', a_est)

In [ ]:
#**********************************************************
# Training the model
#**********************************************************
param_histories = []
cost_histories = np.zeros((n_runs, niter))
error_train_histories = np.zeros((n_runs, niter))

for i in range(n_runs):
    print('---------------------------------')
    print('Start run ', i+1)
    rand_unshaped_pars = rand_params(par_shapes)
    start = time()
    res = my_spsa(get_cost_jit, get_train_error_jit, rand_unshaped_pars,
                  bounds=bounds, niter=niter, a=a_est, c=c_fix,
                  print_iter=False, filename=('RP_task_SPSAOutput_ECS_Run' + str(i)))
    param_histories.append(res[0])   
    cost_histories[i, :] = res[1]
    error_train_histories[i, :] = res[2]    
    print('run', i+1, 'done')
    print('Time taken: ', round(time() - start,2))

In [ ]:
#************************************
# Calculate test errors
#************************************
error_test_histories = np.zeros((n_runs,niter))

for i in range(n_runs):
    test_errors = []
    for params in param_histories[i]:
        test_errors.append(get_test_error_jit(params))
    error_test_histories[i,:] = test_errors

In [ ]:
#************************************
# Calculate average cost and errors
#************************************

cost_history_mean = np.zeros(niter)
error_train_history_mean = np.zeros(niter)
error_test_history_mean = np.zeros(niter)

for i in range(niter):
    cost_history_mean[i] = np.mean(cost_histories[:,i])
    error_train_history_mean[i] = np.mean(error_train_histories[:,i]) 
    error_test_history_mean[i] = np.mean(error_test_histories[:,i]) 

In [ ]:
#****************************************************
# Summary plot
#****************************************************
from matplotlib import pyplot as plt

fig, ax1 = plt.subplots(figsize=(13, 8))

ax1.plot(range(len(cost_history_mean)), cost_history_mean, '-k', markersize=4, label='cost')
ax1.set_ylabel(r"Cost", fontsize='x-large')
ax1.set_xlabel(r"SPSA~iterations", fontsize='x-large')
ax1.legend(loc='upper center', fontsize='x-large')

ax2 = ax1.twinx()
ax2.set_ylabel(r"Error in \%", fontsize='x-large')
ax2.plot(range(len(error_train_history_mean)), error_train_history_mean, '-g', markersize=4, label='train error')
ax2.plot(range(len(error_test_history_mean)), error_test_history_mean, '-b', markersize=4, label='test error')
ax2.legend(loc='upper right', fontsize='x-large')


plt.title('RP task, classical simulation -- results', fontsize='x-large')
plt.savefig('RP_task_ECS_Results.png', dpi=300, facecolor='white')  
plt.show()